In [465]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import regex as re

Step 1

Write the BasicTokenizer class, with the following three core functions:

    def train(self, text, vocab_size, verbose=False)
    def encode(self, text)
    def decode(self, ids)

Let's Train the tokenizer on Taylor Swift's Wikipedia page and Test it on the same text used in the tokenizer lecture

In [466]:
training_data = open("/Users/bhavverma/Documents/karpathy_notes_and_solutions/video_9_dependencies/taylorswift.txt").read()

In [467]:
training_data

'Copy paste of the Wikipedia article on Taylor Swift, as of Feb 16, 2024.\n---\n\nMain menu\n\nWikipediaThe Free Encyclopedia\n\nSearch\nCreate account\nLog in\n\nPersonal tools\nContents  hide\n(Top)\nLife and career\nToggle Life and career subsection\nArtistry\nToggle Artistry subsection\nAccolades and achievements\nCultural status\nToggle Cultural status subsection\nWealth\nToggle Wealth subsection\nDiscography\nFilmography\nTours\nSee also\nFootnotes\nReferences\nToggle References subsection\nExternal links\nTaylor Swift\n\n136 languages\nArticle\nTalk\nRead\nView source\nView history\n\nTools\n Featured article\nPage semi-protected\nFrom Wikipedia, the free encyclopedia\nFor the album, see Taylor Swift (album).\nTaylor Swift\nPortrait of Taylor Swift in a cocktail dress\nSwift at the 2023 MTV Video Music Awards\nBorn\tTaylor Alison Swift\nDecember 13, 1989 (age 34)\nWest Reading, Pennsylvania, US\nOccupations\nSinger-songwriter producer director businesswoman actress\nYears active

In [468]:
test_data = """A Programmer’s Introduction to Unicode March 3, 2017 · Coding · 22 Comments  Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺\u200c🇳\u200c🇮\u200c🇨\u200c🇴\u200c🇩\u200c🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.  A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.  I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.  Diversity and Inherent Complexity The Unicode Codespace Codespace Allocation Scripts Usage Frequency Encodings UTF-8 UTF-16 Combining Marks Canonical Equivalence Normalization Forms Grapheme Clusters And More… Diversity and Inherent Complexity As soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article.  When confronting all this complexity, especially as an engineer, it’s hard not to find oneself asking, “Why do we need all this? Is this really necessary? Couldn’t it be simplified?”  However, Unicode aims to faithfully represent the entire world’s writing systems. The Unicode Consortium’s stated goal is “enabling people around the world to use computers in any language”. And as you might imagine, the diversity of written languages is immense! To date, Unicode supports 135 different scripts, covering some 1100 languages, and there’s still a long tail of over 100 unsupported scripts, both modern and historical, which people are still working to add.  Given this enormous diversity, it’s inevitable that representing it is a complicated project. Unicode embraces that diversity, and accepts the complexity inherent in its mission to include all human writing systems. It doesn’t make a lot of trade-offs in the name of simplification, and it makes exceptions to its own rules where necessary to further its mission.  Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity.  Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity!  The Unicode Codespace Let’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.  The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes.  Codespace Allocation To get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether.  Map of the Unicode codespace (click to zoom)  White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes.  Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more.  (In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.)  Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use.  Scripts Let’s zoom in on the first three planes, since that’s where the action is:  Map of scripts in Unicode planes 0–2 (click to zoom)  This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram.  Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used).  Usage Frequency One more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white.  Heat map of code point usage frequency in Unicode planes 0–2 (click to zoom)  You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1.  Encodings We’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file?  The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.  Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.  Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.  UTF-8 In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.  UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:  UTF-8 (binary)\tCode point (binary)\tRange 0xxxxxxx\txxxxxxx\tU+0000–U+007F 110xxxxx 10yyyyyy\txxxxxyyyyyy\tU+0080–U+07FF 1110xxxx 10yyyyyy 10zzzzzz\txxxxyyyyyyzzzzzz\tU+0800–U+FFFF 11110xxx 10yyyyyy 10zzzzzz 10wwwwww\txxxyyyyyyzzzzzzwwwwww\tU+10000–U+10FFFF A handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.  Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.  However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.  UTF-16 The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.  Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:  UTF-16 (binary)\tCode point (binary)\tRange xxxxxxxxxxxxxxxx\txxxxxxxxxxxxxxxx\tU+0000–U+FFFF 110110xxxxxxxxxx 110111yyyyyyyyyy\txxxxxxxxxxyyyyyyyyyy + 0x10000\tU+10000–U+10FFFF A more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.  Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points.  Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊)  By the way, UTF-16’s words can be stored either little-endian or big-endian. Unicode has no opinion on that issue, though it does encourage the convention of putting U+FEFF zero width no-break space at the top of a UTF-16 file as a byte-order mark, to disambiguate the endianness. (If the file doesn’t match the system’s endianness, the BOM will be decoded as U+FFFE, which isn’t a valid code point.)  Combining Marks In the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point!  Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points.  In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter.  If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called “combining marks”, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character.  For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”.  Now, Unicode does also include many “precomposed” code points, each representing a letter with some combination of diacritics already applied, such as U+00C1 “Á” latin capital letter a with acute or U+1EC7 “ệ” latin small letter e with circumflex and dot below. I suspect these are mostly inherited from older encodings that were assimilated into Unicode, and kept around for compatibility. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text.  Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢n̨̺̝͇͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵s ͘o̱̲͈̙͖͇̲͢n͘ ̜͈e̬̲̠̩ac͕̺̠͉h̷̪ ̺̣͖̱ḻ̫̬̝̹ḙ̙̺͙̭͓̲t̞̞͇̲͉͍t̷͔̪͉̲̻̠͙e̦̻͈͉͇r͇̭̭̬͖,̖́ ̜͙͓̣̭s̘̘͈o̱̰̤̲ͅ ̛̬̜̙t̼̦͕̱̹͕̥h̳̲͈͝ͅa̦t̻̲ ̻̟̭̦̖t̛̰̩h̠͕̳̝̫͕e͈̤̘͖̞͘y҉̝͙ ̷͉͔̰̠o̞̰v͈͈̳̘͜er̶f̰͈͔ḻ͕̘̫̺̲o̲̭͙͠ͅw̱̳̺ ͜t̸h͇̭͕̳͍e̖̯̟̠ ͍̞̜͔̩̪͜ļ͎̪̲͚i̝̲̹̙̩̹n̨̦̩̖ḙ̼̲̼͢ͅ ̬͝s̼͚̘̞͝p͙̘̻a̙c҉͉̜̤͈̯̖i̥͡n̦̠̱͟g̸̗̻̦̭̮̟ͅ ̳̪̠͖̳̯̕a̫͜n͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓  A few other places where dynamic character composition shows up in Unicode:  Vowel-pointing notation in Arabic and Hebrew. In these languages, words are normally spelled with some of their vowels left out. They then have diacritic notation to indicate the vowels (used in dictionaries, language-teaching materials, children’s books, and such). These diacritics are expressed with combining marks.  A Hebrew example, with niqqud:\tאֶת דַלְתִּי הֵזִיז הֵנִיעַ, קֶטֶב לִשְׁכַּתִּי יָשׁוֹד Normal writing (no niqqud):\tאת דלתי הזיז הניע, קטב לשכתי ישוד Devanagari, the script used to write Hindi, Sanskrit, and many other South Asian languages, expresses certain vowels as combining marks attached to consonant letters. For example, “ह” + “\u200bि” = “हि” (“h” + “i” = “hi”). Korean characters stand for syllables, but they are composed of letters called jamo that stand for the vowels and consonants in the syllable. While there are code points for precomposed Korean syllables, it’s also possible to dynamically compose them by concatenating their jamo. For example, “ᄒ” + “ᅡ” + “ᆫ” = “한” (“h” + “a” + “n” = “han”). Canonical Equivalence In Unicode, precomposed characters exist alongside the dynamic composition system. A consequence of this is that there are multiple ways to express “the same” string—different sequences of code points that result in the same user-perceived characters. For example, as we saw earlier, we can express the character “Á” either as the single code point U+00C1, or as the string of two code points U+0041 U+0301.  Another source of ambiguity is the ordering of multiple diacritics in a single character. Diacritic order matters visually when two diacritics apply to the same side of the base character, e.g. both above: “ǡ” (dot, then macron) is different from “ā̇” (macron, then dot). However, when diacritics apply to different sides of the character, e.g. one above and one below, then the order doesn’t affect rendering. Moreover, a character with multiple diacritics might have one of the diacritics precomposed and others expressed as combining marks.  For example, the Vietnamese letter “ệ” can be expressed in five different ways:  Fully precomposed: U+1EC7 “ệ” Partially precomposed: U+1EB9 “ẹ” + U+0302 “◌̂” Partially precomposed: U+00EA “ê” + U+0323 “◌̣” Fully decomposed: U+0065 “e” + U+0323 “◌̣” + U+0302 “◌̂” Fully decomposed: U+0065 “e” + U+0302 “◌̂” + U+0323 “◌̣” Unicode refers to set of strings like this as “canonically equivalent”. Canonically equivalent strings are supposed to be treated as identical for purposes of searching, sorting, rendering, text selection, and so on. This has implications for how you implement operations on text. For example, if an app has a “find in file” operation and the user searches for “ệ”, it should, by default, find occurrences of any of the five versions of “ệ” above!  Normalization Forms To address the problem of “how to handle canonically equivalent strings”, Unicode defines several normalization forms: ways of converting strings into a canonical form so that they can be compared code-point-by-code-point (or byte-by-byte).  The “NFD” normalization form fully decomposes every character down to its component base and combining marks, taking apart any precomposed code points in the string. It also sorts the combining marks in each character according to their rendered position, so e.g. diacritics that go below the character come before the ones that go above the character. (It doesn’t reorder diacritics in the same rendered position, since their order matters visually, as previously mentioned.)  The “NFC” form, conversely, puts things back together into precomposed code points as much as possible. If an unusual combination of diacritics is called for, there may not be any precomposed code point for it, in which case NFC still precomposes what it can and leaves any remaining combining marks in place (again ordered by rendered position, as in NFD).  There are also forms called NFKD and NFKC. The “K” here refers to compatibility decompositions, which cover characters that are “similar” in some sense but not visually identical. However, I’m not going to cover that here.  Grapheme Clusters As we’ve seen, Unicode contains various cases where a thing that a user thinks of as a single “character” might actually be made up of multiple code points under the hood. Unicode formalizes this using the notion of a grapheme cluster: a string of one or more code points that constitute a single “user-perceived character”.  UAX #29 defines the rules for what, precisely, qualifies as a grapheme cluster. It’s approximately “a base code point followed by any number of combining marks”, but the actual definition is a bit more complicated; it accounts for things like Korean jamo, and emoji ZWJ sequences.  The main thing grapheme clusters are used for is text editing: they’re often the most sensible unit for cursor placement and text selection boundaries. Using grapheme clusters for these purposes ensures that you can’t accidentally chop off some diacritics when you copy-and-paste text, that left/right arrow keys always move the cursor by one visible character, and so on.  Another place where grapheme clusters are useful is in enforcing a string length limit—say, on a database field. While the true, underlying limit might be something like the byte length of the string in UTF-8, you wouldn’t want to enforce that by just truncating bytes. At a minimum, you’d want to “round down” to the nearest code point boundary; but even better, round down to the nearest grapheme cluster boundary. Otherwise, you might be corrupting the last character by cutting off a diacritic, or interrupting a jamo sequence or ZWJ sequence.  And More… There’s much more that could be said about Unicode from a programmer’s perspective! I haven’t gotten into such fun topics as case mapping, collation, compatibility decompositions and confusables, Unicode-aware regexes, or bidirectional text. Nor have I said anything yet about implementation issues—how to efficiently store and look-up data about the sparsely-assigned code points, or how to optimize UTF-8 decoding, string comparison, or NFC normalization. Perhaps I’ll return to some of those things in future posts.  Unicode is a fascinating and complex system. It has a many-to-one mapping between bytes and code points, and on top of that a many-to-one (or, under some circumstances, many-to-many) mapping between code points and “characters”. It has oddball special cases in every corner. But no one ever claimed that representing all written languages was going to be easy, and it’s clear that we’re never going back to the bad old days of a patchwork of incompatible encodings.  Further reading:  The Unicode Standard UTF-8 Everywhere Manifesto Dark corners of Unicode by Eevee ICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related things Python 3 Unicode Howto Google Noto Fonts—set of fonts intended to cover all assigned code points"""

In [474]:
#Note \n is its own code point
list(("Aa\na").encode("utf-8"))

[65, 97, 10, 97]

In [485]:
(bytes([65]) + bytes([97])).decode("utf-8")

'Aa'

In [511]:
list((" 1").encode("utf-8"))

[32, 49]

In [490]:
class BasicTokenizer():
    def __init__(self):
        self.vocab = {i: bytes([i]) for i in range(256)} #Need to put i in list for it work
        self.merges = {}
        
    def train(self, text, vocab_size, verbose=False):
        #Get integer encodings of raw bytes
        text = list(text.encode("utf-8"))
        

        while len(self.vocab.keys()) < vocab_size:
            #compute the frequency of byte pairs
            stats = {}
            for ch1, ch2 in zip(text,text[1:]):
                stats[(ch1,ch2)] = stats.get((ch1,ch2), 0) + 1

            #return highest pair
            top_pair = max(stats, key = lambda k: stats[k])

            new_token = len(self.vocab.keys())

            #generate merge history
            self.merges[top_pair] = new_token

            #add new token to vocab, where its value is the concatenation of the byte pair which compose it
            self.vocab[new_token] = self.vocab[top_pair[0]] + self.vocab[top_pair[1]] 
            
            #merge new token 
            new_text = []
            skip_flag = False
            for i in range(len(text)):
                if skip_flag == True:
                    skip_flag = False
                    continue
                if i < len(text) - 1 and (text[i],text[i+1]) == top_pair:
                    new_text.append(new_token)
                    skip_flag = True
                else:
                    new_text.append(text[i])
                    skip_flag = False
            text = new_text

            if verbose == True:
                print(f"pair:{top_pair}, occurances: {stats[top_pair]} --> new token: {new_token}")
            
    def encode(self, text):
        text = list(text.encode("utf-8"))
        #encode whilst new tokens can be merged into text
        any_replacements = True
        if len(text) > 1:
            while any_replacements:
                stats = {}
                for ch1, ch2 in zip(text,text[1:]):
                    stats[(ch1,ch2)] = stats.get((ch1,ch2), 0) + 1

                #return byte pair found in text/tokens which is being encoded that is present in the merges dict and has the lowest token id value
                earliest_merge = min(stats, key = lambda k: self.merges.get(k, float('inf')))
                #if no pairs in merges dict not found, exit
                if earliest_merge not in self.merges.keys():
                    any_replacements = False
                    continue
                
                #merge token found in merges
                new_text = []
                skip_flag = False
                for i in range(len(text)):
                    if skip_flag == True:
                        skip_flag = False
                        continue
                    if i < len(text) - 1 and (text[i],text[i+1]) == earliest_merge:
                        new_text.append(self.merges[earliest_merge])
                        skip_flag = True
                    else:
                        new_text.append(text[i])
                        skip_flag = False
                text = new_text   

        return text

    def decode(self, ids):
        #decode concatenated bytes
        return (b''.join([self.vocab[id] for id in ids])).decode('utf-8', errors = "replace")

In [498]:
b_t = BasicTokenizer()

In [499]:
#Training Tokenizer
b_t.train(training_data, 500, True)

pair:(101, 32), occurances: 2981 --> new token: 256
pair:(44, 32), occurances: 2961 --> new token: 257
pair:(100, 32), occurances: 2617 --> new token: 258
pair:(46, 32), occurances: 2560 --> new token: 259
pair:(114, 32), occurances: 2428 --> new token: 260
pair:(50, 48), occurances: 2365 --> new token: 261
pair:(115, 32), occurances: 2053 --> new token: 262
pair:(105, 110), occurances: 2006 --> new token: 263
pair:(111, 110), occurances: 1815 --> new token: 264
pair:(114, 105), occurances: 1805 --> new token: 265
pair:(116, 32), occurances: 1802 --> new token: 266
pair:(116, 104), occurances: 1737 --> new token: 267
pair:(101, 258), occurances: 1736 --> new token: 268
pair:(257, 261), occurances: 1705 --> new token: 269
pair:(97, 110), occurances: 1487 --> new token: 270
pair:(97, 114), occurances: 1360 --> new token: 271
pair:(101, 260), occurances: 1356 --> new token: 272
pair:(121, 32), occurances: 1248 --> new token: 273
pair:(97, 108), occurances: 1164 --> new token: 274
pair:(26

In [500]:
print(len(b_t.vocab))

500


Now, to determine whether my implementation is equivalent to Andrej's, I added his code to the Notebook and compared the tokenizers: 1) vocab, 2)encoding, and 3) decoding

In [501]:
""" BELOW IS KARPATHY'S CODE FROM MINBPE, USED FOR COMPARISON PURPOSES """

class Tokenizer:
    """Base class for Tokenizers"""

    def __init__(self):
        # default: vocab size of 256 (all bytes), no merges, no patterns
        self.merges = {} # (int, int) -> int
        self.pattern = "" # str
        self.special_tokens = {} # str -> int, e.g. {'<|endoftext|>': 100257}
        self.vocab = self._build_vocab() # int -> bytes

    def train(self, text, vocab_size, verbose=False):
        # Tokenizer can train a vocabulary of size vocab_size from text
        raise NotImplementedError

    def encode(self, text):
        # Tokenizer can encode a string into a list of integers
        raise NotImplementedError

    def decode(self, ids):
        # Tokenizer can decode a list of integers into a string
        raise NotImplementedError

    def _build_vocab(self):
        # vocab is simply and deterministically derived from merges
        vocab = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]
        for special, idx in self.special_tokens.items():
            vocab[idx] = special.encode("utf-8")
        return vocab

    def save(self, file_prefix):
        """
        Saves two files: file_prefix.vocab and file_prefix.model
        This is inspired (but not equivalent to!) sentencepiece's model saving:
        - model file is the critical one, intended for load()
        - vocab file is just a pretty printed version for human inspection only
        """
        # write the model: to be used in load() later
        model_file = file_prefix + ".model"
        with open(model_file, 'w') as f:
            # write the version, pattern and merges, that's all that's needed
            f.write("minbpe v1\n")
            f.write(f"{self.pattern}\n")
            # write the special tokens, first the number of them, then each one
            f.write(f"{len(self.special_tokens)}\n")
            for special, idx in self.special_tokens.items():
                f.write(f"{special} {idx}\n")
            # the merges dict
            for idx1, idx2 in self.merges:
                f.write(f"{idx1} {idx2}\n")
        # write the vocab: for the human to look at
        vocab_file = file_prefix + ".vocab"
        inverted_merges = {idx: pair for pair, idx in self.merges.items()}
        with open(vocab_file, "w", encoding="utf-8") as f:
            for idx, token in self.vocab.items():
                # note: many tokens may be partial utf-8 sequences
                # and cannot be decoded into valid strings. Here we're using
                # errors='replace' to replace them with the replacement char �.
                # this also means that we couldn't possibly use .vocab in load()
                # because decoding in this way is a lossy operation!
                s = render_token(token)
                # find the children of this token, if any
                if idx in inverted_merges:
                    # if this token has children, render it nicely as a merge
                    idx0, idx1 = inverted_merges[idx]
                    s0 = render_token(self.vocab[idx0])
                    s1 = render_token(self.vocab[idx1])
                    f.write(f"[{s0}][{s1}] -> [{s}] {idx}\n")
                else:
                    # otherwise this is leaf token, just print it
                    # (this should just be the first 256 tokens, the bytes)
                    f.write(f"[{s}] {idx}\n")

    def load(self, model_file):
        """Inverse of save() but only for the model file"""
        assert model_file.endswith(".model")
        # read the model file
        merges = {}
        special_tokens = {}
        idx = 256
        with open(model_file, 'r', encoding="utf-8") as f:
            # read the version
            version = f.readline().strip()
            assert version == "minbpe v1"
            # read the pattern
            self.pattern = f.readline().strip()
            # read the special tokens
            num_special = int(f.readline().strip())
            for _ in range(num_special):
                special, special_idx = f.readline().strip().split()
                special_tokens[special] = int(special_idx)
            # read the merges
            for line in f:
                idx1, idx2 = map(int, line.split())
                merges[(idx1, idx2)] = idx
                idx += 1
        self.merges = merges
        self.special_tokens = special_tokens
        self.vocab = self._build_vocab()

class KarpathyBasicTokenizer(Tokenizer):

    def __init__(self):
        super().__init__()

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        # input text preprocessing
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers in range 0..255

        # iteratively merge the most common pairs to create new tokens
        merges = {} # (int, int) -> int
        vocab = {idx: bytes([idx]) for idx in range(256)} # int -> bytes
        for i in range(num_merges):
            # count up the number of times every consecutive pair appears
            stats = get_stats(ids)
            # find the pair with the highest count
            pair = max(stats, key=stats.get)
            # mint a new token: assign it the next available id
            idx = 256 + i
            # replace all occurrences of pair in ids with idx
            ids = merge(ids, pair, idx)
            # save the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            # prints
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurrences")

        # save class variables
        self.merges = merges # used in encode()
        self.vocab = vocab   # used in decode()

    def decode(self, ids):
        # given ids (list of integers), return Python string
        text_bytes = b"".join(self.vocab[idx] for idx in ids)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

    def encode(self, text):
        # given a string text, return the token ids
        text_bytes = text.encode("utf-8") # raw bytes
        ids = list(text_bytes) # list of integers in range 0..255
        while len(ids) >= 2:
            # find the pair with the lowest merge index
            stats = get_stats(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            # subtle: if there are no more merges available, the key will
            # result in an inf for every single pair, and the min will be
            # just the first pair in the list, arbitrarily
            # we can detect this terminating case by a membership check
            if pair not in self.merges:
                break # nothing else can be merged anymore
            # otherwise let's merge the best pair (lowest merge index)
            idx = self.merges[pair]
            ids = merge(ids, pair, idx)
        return ids
           
def get_stats(ids, counts=None):
    """
    Given a list of integers, return a dictionary of counts of consecutive pairs
    Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    Optionally allows to update an existing dictionary of counts
    """
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]): # iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts


def merge(ids, pair, idx):
    """
    In the list of integers (ids), replace all consecutive occurrences
    of pair with the new integer token idx
    Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
    """
    newids = []
    i = 0
    while i < len(ids):
        # if not at the very last position AND the pair matches, replace it
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

# first two helper functions...
def replace_control_characters(s: str) -> str:
    # we don't want to print control characters
    # which distort the output (e.g. \n or much worse)
    # https://stackoverflow.com/questions/4324790/removing-control-characters-from-a-string-in-python/19016117#19016117
    # http://www.unicode.org/reports/tr44/#GC_Values_Table
    chars = []
    for ch in s:
        if unicodedata.category(ch)[0] != "C":
            chars.append(ch) # this character is ok
        else:
            chars.append(f"\\u{ord(ch):04x}") # escape
    return "".join(chars)

def render_token(t: bytes) -> str:
    # pretty print a token, escaping control characters
    s = t.decode('utf-8', errors='replace')
    s = replace_control_characters(s)
    return s

In [506]:
karpathy_b_t = KarpathyBasicTokenizer()
karpathy_b_t.train(training_data, 500)

Let's compare ...

In [507]:
#vocab
b_t.vocab == karpathy_b_t.vocab

True

In [508]:
#encodings
print(b_t.encode(training_data) == karpathy_b_t.encode(training_data))
print(b_t.encode(test_data)== karpathy_b_t.encode(test_data))

True
True


In [509]:
#decodings
print(b_t.decode(b_t.encode(training_data)) == karpathy_b_t.decode(karpathy_b_t.encode(training_data)))
print(b_t.decode(b_t.encode(test_data)) == karpathy_b_t.decode(karpathy_b_t.encode(test_data)))

True
True


STEP 2:

Convert you BasicTokenizer into a RegexTokenizer, which takes a regex pattern and splits the text exactly as GPT-4 would. Process the parts separately as before, then concatenate the results. Retrain your tokenizer and compare the results before and after. You should see that you will now have no tokens that go across categories (numbers, letters, punctuation, more than one whitespace). Use the GPT-4 pattern

In [623]:
from tqdm import tqdm

In [634]:
class RegexTokenizer():
    def __init__(self, pattern):
        self.vocab = {i: bytes([i]) for i in range(256)} 
        self.merges = {}
        self.pattern = pattern
        
    def train(self, text, vocab_size, verbose=False):
        split_text = re.findall(self.pattern, text) #splitting text using Regex pattern
        split_text = [list(text.encode("utf-8")) for text in split_text]

        #accumulating occurances of each byte pair, when considering ALL splits in the original text
        while len(self.vocab.keys()) < vocab_size:
            stats = {}
            for text in split_text:
                if len(text) == 1:
                    continue
                for ch1, ch2 in zip(text,text[1:]):
                    stats[(ch1,ch2)] = stats.get((ch1,ch2), 0) + 1              

            top_pair = max(stats, key = lambda k: stats[k])
            new_token = len(self.vocab.keys())
            self.merges[top_pair] = new_token
            self.vocab[new_token] = self.vocab[top_pair[0]] + self.vocab[top_pair[1]] 
                    
            new_split_text = []
            skip_flag = False
            #going through each split, merging tokens if applicable
            for text in split_text:
                new_text = []
                for i in range(len(text)):
                    if skip_flag == True:
                        skip_flag = False
                        continue
                    if i < len(text) - 1 and (text[i],text[i+1]) == top_pair:
                        new_text.append(new_token)
                        skip_flag = True
                    else:
                        new_text.append(text[i])
                        skip_flag = False
                new_split_text.append(new_text)
            split_text = new_split_text
            if verbose == True:
                print(f"pair:{top_pair}, occurances: {stats[top_pair]} --> new token: {new_token}")
            
    def encode(self, text):
        split_text = re.findall(self.pattern, text) #splitting text using Regex pattern
        split_text = [list(text.encode("utf-8")) for text in split_text]
        encode_text = []
        for text in split_text:
            any_replacements = True
            while any_replacements:
                if len(text) == 1:
                    any_replacements = False
                    continue
                stats = {}
                for ch1, ch2 in zip(text,text[1:]):
                    stats[(ch1,ch2)] = stats.get((ch1,ch2), 1) + 1

                earliest_merge = min(stats, key = lambda k: self.merges.get(k, float('inf')))
                if earliest_merge not in self.merges.keys():
                    any_replacements = False
                    continue

                new_text = []
                skip_flag = False
                for i in range(len(text)):
                    if skip_flag == True:
                        skip_flag = False
                        continue
                    if i < len(text) - 1 and (text[i],text[i+1]) == earliest_merge:
                        new_text.append(self.merges[earliest_merge])
                        skip_flag = True
                    else:
                        new_text.append(text[i])
                        skip_flag = False
                text = new_text  
            encode_text.extend(text) 
        return encode_text

    def decode(self, ids):
        return (b''.join([self.vocab[id] for id in ids])).decode('utf-8', errors = "replace")

In [635]:
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""

In [636]:
r_t = RegexTokenizer(GPT4_SPLIT_PATTERN)

In [637]:
#Training Tokenizer
r_t.train(training_data, 500, True)

pair:(101, 114), occurances: 2359 --> new token: 256
pair:(50, 48), occurances: 2187 --> new token: 257
pair:(111, 114), occurances: 2076 --> new token: 258
pair:(105, 110), occurances: 2006 --> new token: 259
pair:(101, 100), occurances: 1876 --> new token: 260
pair:(32, 116), occurances: 1824 --> new token: 261
pair:(111, 110), occurances: 1815 --> new token: 262
pair:(104, 101), occurances: 1772 --> new token: 263
pair:(32, 83), occurances: 1633 --> new token: 264
pair:(97, 114), occurances: 1519 --> new token: 265
pair:(97, 110), occurances: 1487 --> new token: 266
pair:(32, 65), occurances: 1335 --> new token: 267
pair:(261, 263), occurances: 1169 --> new token: 268
pair:(97, 108), occurances: 1164 --> new token: 269
pair:(114, 105), occurances: 1156 --> new token: 270
pair:(118, 260), occurances: 1104 --> new token: 271
pair:(115, 116), occurances: 1089 --> new token: 272
pair:(119, 105), occurances: 1049 --> new token: 273
pair:(32, 82), occurances: 1045 --> new token: 274
pair:

In [638]:
len(r_t.vocab)

500

Now, to determine whether my implementation is equivalent to Andrej's, I added his code to the Notebook and compared the tokenizers: 1) vocab, 2)encoding, and 3) decoding

In [629]:
""" BELOW IS KARPATHY'S CODE FROM MINBPE, USED FOR COMPARISON PURPOSES """

class KarpathyRegexTokenizer(Tokenizer):

    def __init__(self, pattern=None):
        """
        - pattern: optional string to override the default (GPT-4 split pattern)
        - special_tokens: str -> int dictionary of special tokens
          example: {'<|endoftext|>': 100257}
        """
        super().__init__()
        self.pattern = GPT4_SPLIT_PATTERN if pattern is None else pattern
        self.compiled_pattern = re.compile(self.pattern)
        self.special_tokens = {}
        self.inverse_special_tokens = {}

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        # split the text up into text chunks
        text_chunks = re.findall(self.compiled_pattern, text)

        # input text preprocessing
        ids = [list(ch.encode("utf-8")) for ch in text_chunks]

        # iteratively merge the most common pairs to create new tokens
        merges = {} # (int, int) -> int
        vocab = {idx: bytes([idx]) for idx in range(256)} # idx -> bytes
        for i in range(num_merges):
            # count the number of times every consecutive pair appears
            stats = {}
            for chunk_ids in ids:
                # passing in stats will update it in place, adding up counts
                get_stats(chunk_ids, stats)
            # find the pair with the highest count
            pair = max(stats, key=stats.get)
            # mint a new token: assign it the next available id
            idx = 256 + i
            # replace all occurrences of pair in ids with idx
            ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
            # save the merge
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]
            # prints
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurrences")

        # save class variables
        self.merges = merges # used in encode()
        self.vocab = vocab   # used in decode()

    def register_special_tokens(self, special_tokens):
        # special_tokens is a dictionary of str -> int
        # example: {"<|endoftext|>": 100257}
        self.special_tokens = special_tokens
        self.inverse_special_tokens = {v: k for k, v in special_tokens.items()}

    def decode(self, ids):
        # given ids (list of integers), return Python string
        part_bytes = []
        for idx in ids:
            if idx in self.vocab:
                part_bytes.append(self.vocab[idx])
            elif idx in self.inverse_special_tokens:
                part_bytes.append(self.inverse_special_tokens[idx].encode("utf-8"))
            else:
                raise ValueError(f"invalid token id: {idx}")
        text_bytes = b"".join(part_bytes)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

    def _encode_chunk(self, text_bytes):
        # return the token ids
        # let's begin. first, convert all bytes to integers in range 0..255
        ids = list(text_bytes)
        while len(ids) >= 2:
            # find the pair with the lowest merge index
            stats = get_stats(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            # subtle: if there are no more merges available, the key will
            # result in an inf for every single pair, and the min will be
            # just the first pair in the list, arbitrarily
            # we can detect this terminating case by a membership check
            if pair not in self.merges:
                break # nothing else can be merged anymore
            # otherwise let's merge the best pair (lowest merge index)
            idx = self.merges[pair]
            ids = merge(ids, pair, idx)
        return ids

    def encode_ordinary(self, text):
        """Encoding that ignores any special tokens."""
        # split text into chunks of text by categories defined in regex pattern
        text_chunks = re.findall(self.compiled_pattern, text)
        # all chunks of text are encoded separately, then results are joined
        ids = []
        for chunk in text_chunks:
            chunk_bytes = chunk.encode("utf-8") # raw bytes
            chunk_ids = self._encode_chunk(chunk_bytes)
            ids.extend(chunk_ids)
        return ids

    def encode(self, text, allowed_special="none_raise"):
        """
        Unlike encode_ordinary, this function handles special tokens.
        allowed_special: can be "all"|"none"|"none_raise" or a custom set of special tokens
        if none_raise, then an error is raised if any special token is encountered in text
        this is the default tiktoken behavior right now as well
        any other behavior is either annoying, or a major footgun
        """
        # decode the user desire w.r.t. handling of special tokens
        special = None
        if allowed_special == "all":
            special = self.special_tokens
        elif allowed_special == "none":
            special = {}
        elif allowed_special == "none_raise":
            special = {}
            assert all(token not in text for token in self.special_tokens)
        elif isinstance(allowed_special, set):
            special = {k: v for k, v in self.special_tokens.items() if k in allowed_special}
        else:
            raise ValueError(f"allowed_special={allowed_special} not understood")
        if not special:
            # shortcut: if no special tokens, just use the ordinary encoding
            return self.encode_ordinary(text)
        # otherwise, we have to be careful with potential special tokens in text
        # we handle special tokens by splitting the text
        # based on the occurrence of any exact match with any of the special tokens
        # we can use re.split for this. note that surrounding the pattern with ()
        # makes it into a capturing group, so the special tokens will be included
        special_pattern = "(" + "|".join(re.escape(k) for k in special) + ")"
        special_chunks = re.split(special_pattern, text)
        # now all the special characters are separated from the rest of the text
        # all chunks of text are encoded separately, then results are joined
        ids = []
        for part in special_chunks:
            if part in special:
                # this is a special token, encode it separately as a special case
                ids.append(special[part])
            else:
                # this is an ordinary sequence, encode it normally
                ids.extend(self.encode_ordinary(part))
        return ids

In [639]:
karpathy_r_t = KarpathyRegexTokenizer(GPT4_SPLIT_PATTERN)

In [640]:
karpathy_r_t.train(training_data, 500)

Let's compare ...

In [641]:
#vocab
r_t.vocab == karpathy_r_t.vocab 

True

In [642]:
#encoding
print(r_t.encode(training_data) == karpathy_r_t.encode_ordinary(training_data))
print(r_t.encode(test_data) == karpathy_r_t.encode_ordinary(test_data))

True
True


In [643]:
#decoding
print(r_t.decode(r_t.encode(training_data)) == karpathy_r_t.decode(karpathy_r_t.encode(training_data)))
print(r_t.decode(r_t.encode(test_data)) == karpathy_r_t.decode(karpathy_r_t.encode(test_data)))

True
True


STEP 3

You're now ready to load the merges from the GPT-4 tokenizer and show that your tokenizer produces the identical results for both encode and decode, matching tiktoken.


match this...


import tiktoken

enc = tiktoken.get_encoding("cl100k_base") # this is the GPT-4 tokenizer

ids = enc.encode("hello world!!!? (안녕하세요!) lol123 😉")

text = enc.decode(ids) # get the same text back

In [644]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base") # this is the GPT-4 tokenizer
ids = enc.encode("hello world!!!? (안녕하세요!) lol123 😉")
text = enc.decode(ids) # get the same text back

Will run into two issues...

Issue 1: It is not trivial to recover the raw merges from the GPT-4 tokenizer. You can easily recover what we call vocab here, and what they call and store under enc._mergeable_ranks. Feel free to copy paste the recover_merges function in minbpe/gpt4.py, which takes these ranks and returns the raw merges. If you wish to know how this function works, read this and this. Basically, under some conditions it is enough to only store the parent nodes (and their rank) and get rid of the precise details of which children merged up to any parent.

In [645]:
"Karpathy Code"

def bpe(mergeable_ranks, token, max_rank):
    # helper function used in get_gpt4_merges() to reconstruct the merge forest
    parts = [bytes([b]) for b in token]
    while True:
        min_idx = None
        min_rank = None
        for i, pair in enumerate(zip(parts[:-1], parts[1:])):
            rank = mergeable_ranks.get(pair[0] + pair[1])
            if rank is not None and (min_rank is None or rank < min_rank):
                min_idx = i
                min_rank = rank
        if min_rank is None or (max_rank is not None and min_rank >= max_rank):
            break
        assert min_idx is not None
        parts = parts[:min_idx] + [parts[min_idx] + parts[min_idx + 1]] + parts[min_idx + 2:]
    return parts

def recover_merges(mergeable_ranks):
    # the `merges` are already the byte sequences in their merged state.
    # so we have to recover the original pairings. We can do this by doing
    # a small BPE training run on all the tokens, in their order.
    # also see https://github.com/openai/tiktoken/issues/60
    # also see https://github.com/karpathy/minbpe/issues/11#issuecomment-1950805306
    merges = {}
    for token, rank in mergeable_ranks.items():
        if len(token) == 1:
            continue # skip raw bytes
        pair = tuple(bpe(mergeable_ranks, token, max_rank=rank))
        assert len(pair) == 2
        # recover the integer ranks of the pair
        ix0 = mergeable_ranks[pair[0]]
        ix1 = mergeable_ranks[pair[1]]
        merges[(ix0, ix1)] = rank

    return merges

In [650]:
enc._mergeable_ranks

{b'!': 0,
 b'"': 1,
 b'#': 2,
 b'$': 3,
 b'%': 4,
 b'&': 5,
 b"'": 6,
 b'(': 7,
 b')': 8,
 b'*': 9,
 b'+': 10,
 b',': 11,
 b'-': 12,
 b'.': 13,
 b'/': 14,
 b'0': 15,
 b'1': 16,
 b'2': 17,
 b'3': 18,
 b'4': 19,
 b'5': 20,
 b'6': 21,
 b'7': 22,
 b'8': 23,
 b'9': 24,
 b':': 25,
 b';': 26,
 b'<': 27,
 b'=': 28,
 b'>': 29,
 b'?': 30,
 b'@': 31,
 b'A': 32,
 b'B': 33,
 b'C': 34,
 b'D': 35,
 b'E': 36,
 b'F': 37,
 b'G': 38,
 b'H': 39,
 b'I': 40,
 b'J': 41,
 b'K': 42,
 b'L': 43,
 b'M': 44,
 b'N': 45,
 b'O': 46,
 b'P': 47,
 b'Q': 48,
 b'R': 49,
 b'S': 50,
 b'T': 51,
 b'U': 52,
 b'V': 53,
 b'W': 54,
 b'X': 55,
 b'Y': 56,
 b'Z': 57,
 b'[': 58,
 b'\\': 59,
 b']': 60,
 b'^': 61,
 b'_': 62,
 b'`': 63,
 b'a': 64,
 b'b': 65,
 b'c': 66,
 b'd': 67,
 b'e': 68,
 b'f': 69,
 b'g': 70,
 b'h': 71,
 b'i': 72,
 b'j': 73,
 b'k': 74,
 b'l': 75,
 b'm': 76,
 b'n': 77,
 b'o': 78,
 b'p': 79,
 b'q': 80,
 b'r': 81,
 b's': 82,
 b't': 83,
 b'u': 84,
 b'v': 85,
 b'w': 86,
 b'x': 87,
 b'y': 88,
 b'z': 89,
 b'{': 90,
 b'|': 9

In [653]:
permuted_gpt4_merges = recover_merges(enc._mergeable_ranks)

In [657]:
permuted_gpt4_merges

{(220, 220): 256,
 (256, 256): 257,
 (72, 77): 258,
 (220, 83): 259,
 (257, 257): 260,
 (68, 81): 261,
 (256, 220): 262,
 (78, 77): 263,
 (220, 64): 264,
 (81, 68): 265,
 (64, 83): 266,
 (82, 83): 267,
 (68, 77): 268,
 (78, 81): 269,
 (259, 71): 270,
 (198, 198): 271,
 (220, 66): 272,
 (75, 68): 273,
 (220, 82): 274,
 (72, 83): 275,
 (64, 77): 276,
 (64, 81): 277,
 (64, 75): 278,
 (270, 68): 279,
 (26, 198): 280,
 (220, 79): 281,
 (220, 69): 282,
 (78, 84): 283,
 (220, 28): 284,
 (72, 82): 285,
 (257, 262): 286,
 (258, 70): 287,
 (68, 82): 288,
 (220, 86): 289,
 (72, 263): 290,
 (68, 67): 291,
 (72, 66): 292,
 (220, 65): 293,
 (220, 67): 294,
 (68, 83): 295,
 (220, 76): 296,
 (220, 78): 297,
 (197, 197): 298,
 (81, 78): 299,
 (64, 82): 300,
 (68, 75): 301,
 (66, 83): 302,
 (77, 67): 303,
 (220, 258): 304,
 (220, 71): 305,
 (268, 83): 306,
 (72, 67): 307,
 (220, 77): 308,
 (64, 76): 309,
 (260, 262): 310,
 (259, 78): 311,
 (220, 265): 312,
 (12, 12): 313,
 (220, 90): 314,
 (297, 69): 31

Issue 2:

Second, the GPT-4 tokenizer for some reason permutes its raw bytes. It stores this permutation in the first 256 elements of the mergeable ranks, so you can recover this byte shuffle relatively simply as byte_shuffle = {i: enc._mergeable_ranks[bytes([i])] for i in range(256)}. In both your encode and decode, you'll have to shuffle bytes around accordingly.

In [790]:
class GPTTokenizer():
    def __init__(self, pattern, unshuffle_dict, permuted_vocab, permuted_merge_history):
        self.vocab = {i: bytes([i]) for i in range(256)} 
        self.merges = {}
        self.pattern = pattern
        self.unshuffle_dict = unshuffle_dict #needed to map shuffled byte encodings to original encodings
        self.shuffle_dict = {v:k for k,v in self.unshuffle_dict.items()} #useful for returning encodings which map GPT-4 encodings

        def correct_permutations(coding_point):
            if coding_point in self.unshuffle_dict.keys():
                return self.unshuffle_dict[coding_point]
            else:
                return coding_point
    
        #fixing permutations in byte encodings when creating vocab

        for k,v in permuted_vocab.items():
            self.vocab[correct_permutations(v)] = k 

        #fixing permutations in byte encodings when creating merges history
        for k,v in permuted_merge_history.items():
            k1 = correct_permutations(k[0])
            k2 = correct_permutations(k[1])
            self.merges[tuple([k1,k2])] = v

        
    def train(self, text, vocab_size, verbose=False):
        split_text = re.findall(self.pattern, text) #splitting text using Regex pattern
        split_text = [list(text.encode("utf-8")) for text in split_text]

        #accumulating occurances of each byte pair, when considering ALL splits in the original text
        while len(self.vocab.keys()) < vocab_size:
            stats = {}
            for text in split_text:
                if len(text) == 1:
                    continue
                for ch1, ch2 in zip(text,text[1:]):
                    stats[(ch1,ch2)] = stats.get((ch1,ch2), 0) + 1              

            top_pair = max(stats, key = lambda k: stats[k])
            new_token = len(self.vocab.keys())
            self.merges[top_pair] = new_token
            self.vocab[new_token] = self.vocab[top_pair[0]] + self.vocab[top_pair[1]] 
                    
            new_split_text = []
            skip_flag = False
            #going through each split, merging tokens if applicable
            for text in split_text:
                new_text = []
                for i in range(len(text)):
                    if skip_flag == True:
                        skip_flag = False
                        continue
                    if i < len(text) - 1 and (text[i],text[i+1]) == top_pair:
                        new_text.append(new_token)
                        skip_flag = True
                    else:
                        new_text.append(text[i])
                        skip_flag = False
                new_split_text.append(new_text)
            split_text = new_split_text
            if verbose == True:
                print(f"pair:{top_pair}, occurances: {stats[top_pair]} --> new token: {new_token}")

    def shuffle_idx(self, idx):
        return idx if idx not in self.shuffle_dict.keys() else self.shuffle_dict[idx] 

    def unshuffle_idx(self, idx):
        return idx if idx not in self.unshuffle_dict.keys() else self.unshuffle_dict[idx]
            
    def encode(self, text):
        split_text = re.findall(self.pattern, text) #splitting text using Regex pattern
        split_text = [list(text.encode("utf-8")) for text in split_text]
        encode_text = []
        for text in split_text:
            any_replacements = True
            while any_replacements:
                if len(text) == 1:
                    any_replacements = False
                    continue
                stats = {}
                for ch1, ch2 in zip(text,text[1:]):
                    stats[(ch1,ch2)] = stats.get((ch1,ch2), 1) + 1

                earliest_merge = min(stats, key = lambda k: self.merges.get(k, float('inf')))
                if earliest_merge not in self.merges.keys():
                    any_replacements = False
                    continue

                new_text = []
                skip_flag = False
                for i in range(len(text)):
                    if skip_flag == True:
                        skip_flag = False
                        continue
                    if i < len(text) - 1 and (text[i],text[i+1]) == earliest_merge:
                        new_text.append(self.merges[earliest_merge])
                        skip_flag = True
                    else:
                        new_text.append(text[i])
                        skip_flag = False
                text = new_text  
            encode_text.extend(text) 
        encode_text = [self.shuffle_idx(idx) for idx in encode_text] #For visualizing purposes encode returns the same sets of ids used in GPT4, but these are shuffled weirdly, so when decoding the encoded output, want to remove the shuffle effect
        return encode_text

    def decode(self, ids):
        return (b''.join([self.vocab[self.unshuffle_idx(id)] for id in ids])).decode('utf-8', errors = "replace")

In [791]:
#Create GPT tokenizer
unshuffling_dict = {enc._mergeable_ranks[bytes([i])]: i for i in range(256)}
permute_vocab = enc._mergeable_ranks
permute_merges = recover_merges(enc._mergeable_ranks)

gpt_t = GPTTokenizer(GPT4_SPLIT_PATTERN, unshuffling_dict, permute_vocab, permute_merges)

In [792]:
print(gpt_t.encode("hello world!!!? (안녕하세요!) lol123 😉" ))
print(gpt_t.decode(gpt_t.encode("hello world!!!? (안녕하세요!) lol123 😉")))

[15339, 1917, 12340, 30, 320, 31495, 230, 75265, 243, 92245, 16715, 28509, 4513, 57037]
hello world!!!? (안녕하세요!) lol123 😉


Check if equivalent to TikToken

In [793]:
print(gpt_t.encode("hello world!!!? (안녕하세요!) lol123 😉") == ids)
print(gpt_t.decode(gpt_t.encode("hello world!!!? (안녕하세요!) lol123 😉")) == text)

True
True
